In [1]:
import os
import glob
import atomai as aoi
import matplotlib.pyplot as plt
import numpy as np
import cv2
from scipy import ndimage
from scipy.spatial import distance
import pandas as pd
import json
from tqdm import tqdm

In [2]:
# Define paths
frames_dir = "data/frames"
output_dir = "data/graphs/atomic_positions"
output_metadata_dir = "data/graphs/processing_metadata"

In [3]:
os.makedirs(output_dir, exist_ok=True)
os.makedirs(output_metadata_dir, exist_ok=True)

In [4]:
def process_ws2_image(image_path, target_angstrom_per_pixel=0.12, ws2_lattice_constant=3.15):
    """
    Process WS2 STEM image:
    1. Calculate average distance between tungsten atoms
    2. Resize to achieve target scale (angstroms per pixel)
    3. Normalize intensity to 0-1 range
    
    Parameters:
    - image_path: Path to the WS2 STEM image
    - target_angstrom_per_pixel: Desired resolution in angstroms per pixel
    - ws2_lattice_constant: Known lattice constant of WS2 in angstroms
    
    Returns:
    - processed_img: Resized and normalized image
    - scale_factor: The scaling factor applied
    - avg_distance: Average pixel distance between tungsten atoms
    """
    # Load the image
    original_img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if original_img is None:
        raise FileNotFoundError(f"Could not load image from {image_path}")
    
    # Step 1: Calculate average distance between tungsten atoms
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(original_img, (5, 5), 0)
    
    # Use maximum filter to find local maxima (likely W atoms)
    data_max = ndimage.maximum_filter(blurred, size=5)
    maxima = (blurred == data_max)
    
    # Threshold to eliminate weak maxima
    threshold = 0.5 * np.max(blurred)
    maxima[blurred < threshold] = 0
    
    # Find coordinates of maxima
    labeled, num_objects = ndimage.label(maxima)
    slices = ndimage.find_objects(labeled)
    coordinates = []
    for dy, dx in slices:
        x_center = (dx.start + dx.stop - 1) // 2
        y_center = (dy.start + dy.stop - 1) // 2
        coordinates.append((x_center, y_center))
    
    # Calculate distances between neighboring atoms
    distances = []
    for i, coord1 in enumerate(coordinates):
        # Find the closest points
        coords_array = np.array(coordinates)
        dist_array = distance.cdist([coord1], coords_array)[0]
        
        # Sort distances and skip the first one (distance to itself = 0)
        sorted_indices = np.argsort(dist_array)
        for j in range(1, 7):  # Get the 6 nearest neighbors (hexagonal structure)
            if j < len(sorted_indices):
                neighbor_idx = sorted_indices[j]
                # Only count if it's reasonably close (to avoid false detections)
                if dist_array[neighbor_idx] < 50:  # Adjust threshold as needed
                    distances.append(dist_array[neighbor_idx])
    
    if not distances:
        raise ValueError("Could not detect atomic positions reliably")
    
    # Calculate average distance
    avg_distance = np.mean(distances)
    std_distance = np.std(distances)
    
    print(f"Average distance between tungsten atoms: {avg_distance:.2f} pixels")
    print(f"Standard deviation: {std_distance:.2f} pixels")
    
    # Step 2: Calculate current scale and resize the image
    current_angstrom_per_pixel = ws2_lattice_constant / avg_distance
    scale_factor = current_angstrom_per_pixel / target_angstrom_per_pixel
    
    print(f"Current scale: {current_angstrom_per_pixel:.4f} angstroms/pixel")
    print(f"Target scale: {target_angstrom_per_pixel} angstroms/pixel")
    print(f"Scale factor: {scale_factor:.4f}")
    
    # Resize the image
    new_width = int(original_img.shape[1] * scale_factor)
    new_height = int(original_img.shape[0] * scale_factor)
    resized_img = cv2.resize(original_img, (new_width, new_height), interpolation=cv2.INTER_CUBIC)
    
    # Step 3: Normalize the intensity to 0-1 range
    resized_float = resized_img.astype(np.float32)
    normalized_img = (resized_float - np.min(resized_float)) / (np.max(resized_float) - np.min(resized_float))
    
    return normalized_img, scale_factor, avg_distance

In [5]:
def process_single_frame(frame_path, model, ensemble, model_idx=None):
    """Process a single frame and extract atomic positions."""
    print(f"Processing frame: {os.path.basename(frame_path)}")
    
    # Process the image
    img, scale_factor, avg_distance = process_ws2_image(frame_path)
    
    # Apply the selected model to the frame
    nn_out, _ = aoi.predictors.predictor.SegPredictor(model, use_gpu=True).run(img)
    
    # You can also apply all ensemble models if needed:
    ensemble_outputs = []
    if ensemble is not None:
        for i, m in enumerate(ensemble.values()):
            model.load_state_dict(m)
            out, _ = aoi.predictors.predictor.SegPredictor(model, use_gpu=True).run(img)
            ensemble_outputs.append(out)
    
    return {
        'output': nn_out,
        'scale_factor': scale_factor,
        'avg_distance': avg_distance,
        'ensemble_outputs': ensemble_outputs
    }

In [6]:
def extract_atomic_positions(prediction_output, threshold=0.33):
    """Extract atomic positions from the neural network output."""
    # Apply threshold to get binary mask
    binary_mask = prediction_output[0,...,0] > threshold
    
    # Find connected components
    labeled, num_features = ndimage.label(binary_mask)
    
    # Get positions as center of mass of each component
    positions = ndimage.center_of_mass(binary_mask, labeled, range(1, num_features + 1))
    
    # Convert to format: [y, x] coordinates
    positions = np.array(positions)
    
    return positions

In [7]:
def save_positions(positions, frame_id, output_dir):
    """Save atomic positions to files."""
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # Save as numpy array
    npy_path = os.path.join(output_dir, f"frame_{frame_id:05d}_positions.npy")
    print(f"Saving numpy array to: {npy_path}")
    np.save(npy_path, positions)
    
    # Save as CSV for easy inspection
    csv_path = os.path.join(output_dir, f"frame_{frame_id:05d}_positions.csv")
    print(f"Saving CSV to: {csv_path}")
    df = pd.DataFrame(positions, columns=['y', 'x'])
    df['frame_id'] = frame_id
    df.to_csv(csv_path, index=False)
    
    # Save as JSON (alternative format)
    json_path = os.path.join(output_dir, f"frame_{frame_id:05d}_positions.json")
    print(f"Saving JSON to: {json_path}")
    positions_list = positions.tolist()
    json_data = {
        'frame_id': frame_id,
        'positions': positions_list,
        'num_atoms': len(positions)
    }
    with open(json_path, 'w') as f:
        json.dump(json_data, f, indent=4)

In [8]:
def save_visualization(original_img, nn_out, positions, frame_id, output_dir):
    """Save visualization of the detected atomic positions."""
    plt.figure(figsize=(15, 5))
    
    # Original image
    plt.subplot(131)
    plt.imshow(original_img, origin="lower", cmap='gray')
    plt.title(f"Original Frame {frame_id}")
    plt.axis('off')
    
    # Neural network output
    plt.subplot(132)
    plt.imshow(nn_out[0,...,0], origin="lower", cmap='jet', interpolation="gaussian")
    plt.title(f"NN Output Frame {frame_id}")
    plt.axis('off')
    
    # Detected positions overlaid on original
    plt.subplot(133)
    plt.imshow(original_img, origin="lower", cmap='gray')
    if len(positions) > 0:
        plt.scatter(positions[:, 1], positions[:, 0], c='red', s=5, alpha=0.7, marker='o')
    plt.title(f"Detected Atoms Frame {frame_id}")
    plt.axis('off')
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"frame_{frame_id:05d}_visualization.png"), dpi=300, bbox_inches='tight')
    plt.close()

In [9]:
def process_all_frames(frames_dir, ensemble_path, output_dir):
    """Process all frames in the specified directory."""
    
    # Load the ensemble model
    print("Loading ensemble model...")
    smodel, ensemble = aoi.models.load_ensemble(ensemble_path)
    
    # Choose which ensemble member to use (e.g., model 4)
    selected_model_idx = 4
    smodel.load_state_dict(ensemble[selected_model_idx])
    
    # Get list of all frame files
    frame_extensions = ['*.png', '*.jpg', '*.jpeg', '*.tif', '*.tiff']
    frame_files = []
    for ext in frame_extensions:
        frame_files.extend(glob.glob(os.path.join(frames_dir, ext)))
    
    frame_files.sort()  # Sort for consistent ordering
    
    print(f"Found {len(frame_files)} frames to process")
    
    # Process each frame
    all_positions = {}
    processing_metadata = []
    
    for frame_id, frame_path in enumerate(tqdm(frame_files, desc="Processing frames")):
        try:
            # Process the frame
            result = process_single_frame(frame_path, smodel, ensemble)
            
            # Extract positions
            positions = extract_atomic_positions(result['output'])
            
            # Save positions
            save_positions(positions, frame_id, output_dir)
            
            # Store in memory for later analysis
            all_positions[frame_id] = positions
            
            # Save visualization
            original_img = cv2.imread(frame_path, cv2.IMREAD_GRAYSCALE)
            if original_img is not None:
                save_visualization(original_img, result['output'], positions, frame_id, output_dir)
            
            # Save metadata
            metadata = {
                'frame_id': frame_id,
                'frame_path': frame_path,
                'scale_factor': result['scale_factor'],
                'avg_distance': result['avg_distance'],
                'num_atoms': len(positions),
                'status': 'success'
            }
            processing_metadata.append(metadata)
            
        except Exception as e:
            print(f"Error processing frame {frame_id}: {str(e)}")
            metadata = {
                'frame_id': frame_id,
                'frame_path': frame_path,
                'status': 'error',
                'error': str(e)
            }
            processing_metadata.append(metadata)
    # Save all positions to a single file for convenience
    np.save(os.path.join(output_dir, "all_positions.npy"), all_positions)
    
    # Save processing metadata
    metadata_df = pd.DataFrame(processing_metadata)
    metadata_df.to_csv(os.path.join(output_metadata_dir, "processing_metadata.csv"), index=False)
    
    print(f"Processing complete! Results saved to {output_dir}")
    
    return all_positions, metadata_df

In [10]:
ensemble_path = "data/graphs/models/mos2_4x_ensemble_metadict.tar"

In [11]:
all_positions, metadata = process_all_frames(frames_dir, ensemble_path, output_dir)

# Basic statistics
print("\nProcessing Summary:")
print(f"Total frames processed: {len(metadata)}")
print(f"Successful: {len(metadata[metadata['status'] == 'success'])}")
print(f"Failed: {len(metadata[metadata['status'] == 'error'])}")

# Example: access positions for a specific frame
frame_0_positions = all_positions[0]
print(f"\nFrame 0 detected {len(frame_0_positions)} atoms")

Loading ensemble model...
Found 100 frames to process


Processing frames:   0%|          | 0/100 [00:00<?, ?it/s]

Processing frame: frame_00000.png
Average distance between tungsten atoms: 9.80 pixels
Standard deviation: 1.61 pixels
Current scale: 0.3215 angstroms/pixel
Target scale: 0.12 angstroms/pixel
Scale factor: 2.6794
Batch 1/1
1 image was decoded in approximately 0.1262 seconds
Batch 1/1
1 image was decoded in approximately 0.0781 seconds
Batch 1/1
1 image was decoded in approximately 0.0744 seconds
Batch 1/1
1 image was decoded in approximately 0.0771 seconds
Batch 1/1
1 image was decoded in approximately 0.077 seconds
Batch 1/1
1 image was decoded in approximately 0.0722 seconds
Batch 1/1
1 image was decoded in approximately 0.0782 seconds
Batch 1/1
1 image was decoded in approximately 0.08 seconds
Batch 1/1
1 image was decoded in approximately 0.0789 seconds
Batch 1/1
1 image was decoded in approximately 0.0794 seconds
Batch 1/1
1 image was decoded in approximately 0.0786 seconds
Saving numpy array to: data/graphs/atomic_positions/frame_00000_positions.npy
Saving CSV to: data/graphs/ato

Processing frames:   1%|          | 1/100 [00:04<07:13,  4.38s/it]

Processing frame: frame_00001.png
Average distance between tungsten atoms: 9.78 pixels
Standard deviation: 1.64 pixels
Current scale: 0.3219 angstroms/pixel
Target scale: 0.12 angstroms/pixel
Scale factor: 2.6829
Batch 1/1
1 image was decoded in approximately 0.0711 seconds
Batch 1/1
1 image was decoded in approximately 0.0773 seconds
Batch 1/1
1 image was decoded in approximately 0.0749 seconds
Batch 1/1
1 image was decoded in approximately 0.0727 seconds
Batch 1/1
1 image was decoded in approximately 0.0739 seconds
Batch 1/1
1 image was decoded in approximately 0.0755 seconds
Batch 1/1
1 image was decoded in approximately 0.0746 seconds
Batch 1/1
1 image was decoded in approximately 0.0679 seconds
Batch 1/1
1 image was decoded in approximately 0.0868 seconds
Batch 1/1
1 image was decoded in approximately 0.07 seconds
Batch 1/1
1 image was decoded in approximately 0.0724 seconds
Saving numpy array to: data/graphs/atomic_positions/frame_00001_positions.npy
Saving CSV to: data/graphs/at

...

Processing frame: frame_00098.png
Average distance between tungsten atoms: 12.63 pixels
Standard deviation: 7.19 pixels
Current scale: 0.2494 angstroms/pixel
Target scale: 0.12 angstroms/pixel
Scale factor: 2.0784
Batch 1/1
1 image was decoded in approximately 0.1241 seconds
Batch 1/1
1 image was decoded in approximately 0.0739 seconds
Batch 1/1
1 image was decoded in approximately 0.0477 seconds
Batch 1/1
1 image was decoded in approximately 0.0475 seconds
Batch 1/1
1 image was decoded in approximately 0.0467 seconds
Batch 1/1
1 image was decoded in approximately 0.045 seconds
Batch 1/1
1 image was decoded in approximately 0.0478 seconds
Batch 1/1
1 image was decoded in approximately 0.046 seconds
Batch 1/1
1 image was decoded in approximately 0.0451 seconds
Batch 1/1
1 image was decoded in approximately 0.0462 seconds
Batch 1/1
1 image was decoded in approximately 0.0448 seconds
Saving numpy array to: data/graphs/atomic_positions/frame_00098_positions.npy
Saving CSV to: data/graphs/a

Processing frames:  99%|█████████▉| 99/100 [05:26<00:02,  2.82s/it]

Processing frame: frame_00099.png
Average distance between tungsten atoms: 13.43 pixels
Standard deviation: 7.67 pixels
Current scale: 0.2345 angstroms/pixel
Target scale: 0.12 angstroms/pixel
Scale factor: 1.9544
Batch 1/1
1 image was decoded in approximately 0.0431 seconds
Batch 1/1
1 image was decoded in approximately 0.0429 seconds
Batch 1/1
1 image was decoded in approximately 0.0425 seconds
Batch 1/1
1 image was decoded in approximately 0.0431 seconds
Batch 1/1
1 image was decoded in approximately 0.043 seconds
Batch 1/1
1 image was decoded in approximately 0.0429 seconds
Batch 1/1
1 image was decoded in approximately 0.0471 seconds
Batch 1/1
1 image was decoded in approximately 0.0434 seconds
Batch 1/1
1 image was decoded in approximately 0.0433 seconds
Batch 1/1
1 image was decoded in approximately 0.0421 seconds
Batch 1/1
1 image was decoded in approximately 0.0428 seconds
Saving numpy array to: data/graphs/atomic_positions/frame_00099_positions.npy
Saving CSV to: data/graphs/

Processing frames: 100%|██████████| 100/100 [05:28<00:00,  3.28s/it]

Processing complete! Results saved to data/graphs/atomic_positions

Processing Summary:
Total frames processed: 100
Successful: 100
Failed: 0

Frame 0 detected 6756 atoms
